In [40]:
%pip install pycryptodome

Note: you may need to restart the kernel to use updated packages.


In [41]:
import socket
import datetime
from dateutil import parser
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from base64 import b64encode, b64decode
import hashlib
import os
from random import randrange, getrandbits
from math import gcd
import json

In [42]:
def is_prime(n, k=8):
    """ Test if a number is prime
        Args:
            n -- int -- the number to test
            k -- int -- the number of tests to do
        return True if n is prime
    """
    # Test if n is not even.
    # But care, 2 is prime !
    if n == 2 or n == 3:
        return True
    if n <= 1 or n % 2 == 0:
        return False
    # find r and s
    s = 0
    r = n - 1
    while r & 1 == 0:
        s += 1
        r //= 2
    # do k tests
    for _ in range(k):
        a = randrange(2, n - 1)
        x = pow(a, r, n)
        if x != 1 and x != n - 1:
            j = 1
            while j < s and x != n - 1:
                x = pow(x, 2, n)
                if x == 1:
                    return False
                j += 1
            if x != n - 1:
                return False
    return True

def generate_prime_candidate(length):
    """ Generate an odd integer randomly
        Args:
            length -- int -- the length of the number to generate, in bits
        return a integer
    """
    # generate random bits
    p = getrandbits(length)
    # apply a mask to set MSB and LSB to 1
    p |= (1 << length - 1) | 1
    return p

def generate_prime_number(prev, length=8):
    """ Generate a prime
        Args:
            length -- int -- length of the prime to generate, in bits
        return a prime
    """
    p = 4
    # keep generating while the primality test fail
    while not is_prime(p, 4) or p == prev:
        p = generate_prime_candidate(length)
    return p


def generate_public_key(phi):
    e = 2
    while True:
        if gcd(e, phi) == 1:
            return e
        e += 1

def generate_rsa_key():
    p = generate_prime_number(1,8)
    q = generate_prime_number(p,8)
    n = p * q
    phi = (p-1) * (q-1)
    e = generate_public_key(phi)
    d = pow(e, -1, phi)
    return (e, n), (d, n)


def encrypt_rsa(pk, plaintext):
    # Unpack the key into it's components
    key, n = pk
    # Convert each letter in the plaintext to numbers based on the character using a^b mod m
    cipher = [pow(ord(char), key, n) for char in plaintext]
    # Return the array of bytes
    return cipher


def decrypt_rsa(pk, ciphertext):
    # Unpack the key into its components
    key, n = pk
    # Generate the plaintext based on the ciphertext and key using a^b mod m
    plain = [chr(pow(char, key, n)) for char in ciphertext]
    # Return the array of bytes as a string
    return ''.join(plain)


In [43]:
def generate_id():
    return os.urandom(16).hex()

In [44]:
def add_client_info(public_key, key_ring, client_id):
    key_ring[client_id] = public_key
    
def get_client_info(key_ring, client_id):
    return key_ring[client_id]

In [45]:
def add_symmetric_key(symmetric_key, key_ring, client_id):
    key_ring[client_id] = symmetric_key

def get_symmetric_key(key_ring, client_id):
    return key_ring[client_id]

In [46]:
def encrypt_AES(data, key):
    cipher = AES.new(key, AES.MODE_ECB)
    ct_bytes = cipher.encrypt(pad(data, AES.block_size))
    ct = b64encode(ct_bytes).decode('utf-8')
    return ct

def decrypt_AES(ct, key):
    cipher = AES.new(key, AES.MODE_ECB)
    pt = unpad(cipher.decrypt(b64decode(ct)), AES.block_size)
    return pt


In [47]:
def get_file_hash(data):
    return hashlib.sha512(data).hexdigest() 

def get_file_hash_from_encrypted_digest(server_public_key, encrypted_file_digest):
    return decrypt_rsa(server_public_key, encrypted_file_digest)

def get_file_bytes(file):
    return b64decode(file)

In [48]:
def handshake_with_KDC(kdc_connection, id, public_key, public_key_ring):
    #send handshake message to KDC
    handshake_message = {
        'id': id,
        'public_key': public_key
    }
    handshake_message_str = json.dumps(handshake_message)
    kdc_connection.send(str.encode(handshake_message_str))

    #receive handshake response from KDC
    handshake_response_str = kdc_connection.recv(2048).decode('utf-8')
    handshake_response = json.loads(handshake_response_str)
    add_client_info(handshake_response['public_key'], public_key_ring, handshake_response['id'])
    print('Handshake with KDC successful')

In [49]:
def request_server_public_key_from_KDC(kdc_connection, id, public_key_ring, private_key):
    #send public key request for degree granting server to KDC
    message_payload = {
        'requested_id': 'degree-granting-server'
    }
    encrypted_message_payload = encrypt_rsa(private_key, json.dumps(message_payload))
    public_key_request_message = {
        'id': id,
        'encrypted_payload': encrypted_message_payload,
    }
    kdc_connection.send(str.encode(json.dumps(public_key_request_message)))

    #receive public key for degree granting server from KDC
    public_key_response_str = kdc_connection.recv(2048).decode('utf-8')
    public_key_response = json.loads(public_key_response_str)
    encrypted_payload = public_key_response['encrypted_payload']
    decrypted_payload_str = decrypt_rsa(private_key, encrypted_payload)
    decrypted_payload = json.loads(decrypted_payload_str)
    if(decrypted_payload['requested_id'] != 'degree-granting-server'):
        print('Error: received public key for wrong server')
        return
    else:
        add_client_info(decrypted_payload['requested_public_key'], public_key_ring, decrypted_payload['requested_id'])
    print('Received public key for degree granting server from KDC')

In [50]:
def send_authentication_request_to_server(certificate_connection, name, roll_number, password, id, server_public_key, private_key, private_key_ring):
    message_payload = {
        'name': name,
        'roll_number': roll_number,
        'password': password
    }
    encrypted_message_payload = encrypt_rsa(server_public_key, json.dumps(message_payload))
    authentication_request_message = {
        'id': id,
        'encrypted_payload': encrypted_message_payload,
    }
    certificate_connection.send(str.encode(json.dumps(authentication_request_message)))

    #receive symmetric key from degree granting server
    symmetric_key_response_str = certificate_connection.recv(2048).decode('utf-8')
    symmetric_key_response = json.loads(symmetric_key_response_str)
    encrypted_payload = symmetric_key_response['encrypted_payload']
    decrypted_payload_str = decrypt_rsa(private_key, encrypted_payload)
    decrypted_payload = json.loads(decrypted_payload_str)
    symmetric_key = bytes.fromhex(decrypted_payload['symmetric_key'])
    add_symmetric_key(symmetric_key, private_key_ring, 'degree-granting-server')
    print('Received symmetric key from degree granting server {}'.format(symmetric_key))
    

In [51]:
def send_degree_request_to_server(certificate_connection, name, roll_number, private_key_ring, id, public_key_ring):
    message_payload = {
        'name': name,
        'roll_number': roll_number,
    }
    # convert json object to bytes
    encrypted_message_payload = encrypt_AES(str.encode(json.dumps(message_payload)), get_symmetric_key(private_key_ring, 'degree-granting-server'))
    degree_request_message = {
        'id': id,
        'encrypted_payload': encrypted_message_payload,
    }
    print('Encrypted message payload: {}'.format(encrypted_message_payload))
    certificate_connection.send(str.encode(json.dumps(degree_request_message)))

    #receive degree from degree granting server
    while True:
        degree_response = certificate_connection.recv(2048).decode('utf-8')
        if not degree_response:
            break
        degree_response = json.loads(degree_response)
        encrypted_payload = degree_response['encrypted_payload']
        decrypted_payload = decrypt_AES(encrypted_payload, get_symmetric_key(private_key_ring, 'degree-granting-server'))
        decrypted_payload = json.loads(decrypted_payload.decode('utf-8'))
        file_digest = decrypted_payload['file_digest']
        file_data = get_file_bytes(decrypted_payload['file'])
        received_file_hash = get_file_hash(file_data)
        calculated_file_hash = get_file_hash_from_encrypted_digest(get_client_info(public_key_ring, 'degree-granting-server'), file_digest)
        if received_file_hash != calculated_file_hash:
            print('Error: received file hash does not match calculated file hash')
            return
        else:
            print('Received file hash matches calculated file hash')
            with open('{}_grade_card_downloaded.txt'.format(roll_number), 'wb') as f:
                f.write(file_data)
            return

In [52]:
def get_synchronized_time(clock_server_connection):
    #Time at which client sent a request to clockServer
    clock_server_connection.send("SYNC_TIME".encode('utf-8'))
    clock_server_request_time = datetime.datetime.now().timestamp()   #T0
    synced_time_str = clock_server_connection.recv(2048).decode()
    clock_data_recv = parser.parse(synced_time_str)
    #Time at which client received a response from the clockServer
    clock_server_response_time = datetime.datetime.now().timestamp() #T1
    latency = clock_server_response_time - clock_server_request_time
    client_time = clock_data_recv + datetime.timedelta(seconds= latency/2)
    return client_time

In [53]:
host = '127.0.0.1'
clock_port = 8080
certificate_port = 5000
kdc_port = 6666

# clock_server_connection = socket.socket()
# clock_server_connection.connect((host, clock_port))

kdc_connection = socket.socket()
kdc_connection.connect((host, kdc_port))


#connects to certificate granting server
# synced_time = get_synchronized_time(clock_server_connection)
# print('Synced time', synced_time)
# clock_server_connection.close()

id = generate_id()
public_key, private_key = generate_rsa_key()
public_key_ring = {}
private_key_ring = {}
handshake_with_KDC(kdc_connection, id, public_key, public_key_ring)
request_server_public_key_from_KDC(kdc_connection, id, public_key_ring, private_key)

certificate_server_connection = socket.socket()
certificate_server_connection.connect((host, certificate_port))

inputName = input("Enter your name: ")
inputRollNumber = input("Enter your roll number: ")
inputPassword = input("Enter your password: ")

send_authentication_request_to_server(certificate_server_connection, inputName, inputRollNumber, inputPassword, id, get_client_info(public_key_ring, 'degree-granting-server'), private_key, private_key_ring)
send_degree_request_to_server(certificate_server_connection, inputName, inputRollNumber, private_key_ring, id, public_key_ring)








Handshake with KDC successful
Received public key for degree granting server from KDC
Received symmetric key from degree granting server b'\xd2~\xdag$\xb7\xca\xf8\x8e\x9f\xfax\xb5\x01Ds'
Encrypted message payload: zTVpVMKjIs1j1JXUx7rkf+csHRJ/aI092kJA570L2ShSaRBn6EIt/B7BJjb5zHvv


JSONDecodeError: Unterminated string starting at: line 1 column 55 (char 54)